## import env values

In [12]:
from dotenv import load_dotenv

load_dotenv("../docker/.env")

True

In [13]:
import os

In [14]:
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_DB = os.getenv('DB_DB')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

## extract data from postgres

In [17]:
import psycopg2
import pandas as pd

def get_table(table_name: str, db: str, user: str, password: str, host: str, port: int) -> pd.DataFrame: # postgres operator?
    conn = None
    try:
        conn_string = "dbname=%s user=%s password=%s host=%s port=%s" % (db,user,password,host,port) # AWS Secret Manager
        conn = psycopg2.connect(conn_string)
        cur = conn.cursor()
        sql = "SELECT * FROM " + table_name
        df = pd.read_sql_query(sql,conn)

        return df
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

df = get_table("reviews", DB_DB, DB_USER, DB_PASSWORD, DB_HOST, DB_PORT) # REVIEW WARNING

In [6]:
import datetime

In [7]:
df_manual['inserted_at'] = datetime.datetime.now()

## write data to delta lake

In [91]:
from deltalake import write_deltalake
storage_options = {"AWS_ACCESS_KEY_ID": ACCESS_KEY, "AWS_SECRET_ACCESS_KEY": SECRET_KEY, "AWS_REGION": "us-east-1", "AWS_S3_ALLOW_UNSAFE_RENAME": "true"}
write_deltalake('s3://b-data-stg/1-raw/assessments/reviews/', df, storage_options=storage_options, mode='append')

Exception: Schema error: Invalid data type for Delta Lake: Null

## read data from delta lake

In [111]:
from deltalake import DeltaTable
dt = DeltaTable('s3://b-data-stg/raw/assessments/reviews/', storage_options=storage_options)

In [112]:
dt.to_pandas()

,id,assessment_id,primary_verification_lead,secondary_verification_lead,opened_date,closed_date,closed_reason,closed_details,evaluation_analyst,closed_by,...,review_sfid,survey_sent_at,review_call_language_preferences,is_recertification,cost_plus_country,cost_plus_global_partner,cost_plus_points,scheduled_removal_of_certification_team_leads_at,completed_removal_of_certification_team_leads_at,last_task_based_email_sent_at
0,21810,190823,199549.0,NaN,2020-02-04 01:28:07.372788,2021-03-18 02:52:34.840711,completed-b-corp-certification,Re-certification completed for the next 3-year...,49233.0,199549.0,...,a5t3b000001W0p4AAC,2021-03-11 04:39:09.047627,{en-us},True,United States,B Lab US/Canada,0.75,,NaT,NaT
1,30621,212773,148549.0,NaN,2020-08-19 10:21:23.099853,2021-03-18 13:54:55.363612,completed-b-corp-certification,"With a finalized score of 123.2, welcome back ...",49233.0,148549.0,...,a5t3b0000000HxkAAE,2021-03-17 14:42:59.528756,{en-us},True,Italy,B Lab Europe,0.75,,NaT,NaT
2,29633,212590,177552.0,NaN,2020-05-11 00:38:33.958234,2021-03-24 14:11:04.223464,completed-b-corp-certification,Welcome to the B Corp community!,49242.0,177552.0,...,a5t3b000001WXwjAAG,2021-03-12 19:42:30.200710,{en-us},False,Australia,B Lab Australia/NZ,1.00,,NaT,NaT
3,36020,282721,NaN,NaN,2021-06-04 05:39:26.569911,2021-11-19 18:12:04.713247,not-engaged,The required tasks were not completed by the d...,NaN,84064.0,...,a5t3b000001R7bIAAS,NaT,{fr-fr},False,Canada,B Lab US/Canada,1.00,,NaT,NaT
4,37497,301949,NaN,NaN,2021-09-20 17:27:09.333299,2021-12-10 15:18:10.362204,ineligible,Company did not respond to outreach over 2 mon...,NaN,240476.0,...,a5t3b000001YgyQAAS,NaT,{en-us},False,United States,B Lab US/Canada,0.10,,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63657,44799,386290,NaN,NaN,2023-06-15 14:37:20.401212,NaT,,,NaN,NaN,...,a5t3b000001WfI2AAK,NaT,"{fr-fr,en-us}",False,,,NaN,,NaT,2023-09-14
63658,39412,306231,336557.0,243846.0,2022-03-09 14:10:51.217884,NaT,,,243704.0,NaN,...,a5t3b000000AGJJAA4,2023-05-10 20:55:37.981692,{other},False,,,NaN,,NaT,2023-06-16
63659,43410,359825,NaN,NaN,2023-02-17 13:02:39.892716,NaT,,,249112.0,NaN,...,a5t3b000001ROP5AAO,NaT,{en-us},False,,,NaN,,NaT,2023-06-30
63660,36867,396643,243853.0,NaN,2021-07-30 08:08:12.450729,2023-06-12 08:59:10.761784,completed-b-corp-certification,Congratulations!,249112.0,243853.0,...,a5t3b000001RBF1AAO,2023-06-01 07:26:15.438373,{en-us},False,United Kingdom,B Lab UK,2.00,,2023-12-13 02:30:37.992040,2023-06-02


In [ ]:
dt.history()

In [18]:
from deltalake import DeltaTable
from deltalake import DataCatalog
database_name = "b_data_stg"
table_name = "assessments_reviews"
data_catalog = DataCatalog.AWS
dt = DeltaTable.from_data_catalog(data_catalog=data_catalog, database_name=database_name, table_name=table_name)

In [20]:
dt.to_pandas()

,id,assessment_id,primary_verification_lead,secondary_verification_lead,opened_date,closed_date,closed_reason,closed_details,evaluation_analyst,closed_by,...,review_sfid,survey_sent_at,review_call_language_preferences,is_recertification,cost_plus_country,cost_plus_global_partner,cost_plus_points,scheduled_removal_of_certification_team_leads_at,completed_removal_of_certification_team_leads_at,last_task_based_email_sent_at
0,21810,190823,199549.0,NaN,2020-02-04 01:28:07.372788,2021-03-18 02:52:34.840711,completed-b-corp-certification,Re-certification completed for the next 3-year...,49233.0,199549.0,...,a5t3b000001W0p4AAC,2021-03-11 04:39:09.047627,{en-us},True,United States,B Lab US/Canada,0.75,NaT,NaT,NaT
1,30621,212773,148549.0,NaN,2020-08-19 10:21:23.099853,2021-03-18 13:54:55.363612,completed-b-corp-certification,"With a finalized score of 123.2, welcome back ...",49233.0,148549.0,...,a5t3b0000000HxkAAE,2021-03-17 14:42:59.528756,{en-us},True,Italy,B Lab Europe,0.75,NaT,NaT,NaT
2,29633,212590,177552.0,NaN,2020-05-11 00:38:33.958234,2021-03-24 14:11:04.223464,completed-b-corp-certification,Welcome to the B Corp community!,49242.0,177552.0,...,a5t3b000001WXwjAAG,2021-03-12 19:42:30.200710,{en-us},False,Australia,B Lab Australia/NZ,1.00,NaT,NaT,NaT
3,36020,282721,NaN,NaN,2021-06-04 05:39:26.569911,2021-11-19 18:12:04.713247,not-engaged,The required tasks were not completed by the d...,NaN,84064.0,...,a5t3b000001R7bIAAS,NaT,{fr-fr},False,Canada,B Lab US/Canada,1.00,NaT,NaT,NaT
4,37497,301949,NaN,NaN,2021-09-20 17:27:09.333299,2021-12-10 15:18:10.362204,ineligible,Company did not respond to outreach over 2 mon...,NaN,240476.0,...,a5t3b000001YgyQAAS,NaT,{en-us},False,United States,B Lab US/Canada,0.10,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31826,44799,386290,NaN,NaN,2023-06-15 14:37:20.401212,NaT,None,None,NaN,NaN,...,a5t3b000001WfI2AAK,NaT,"{fr-fr,en-us}",False,None,None,NaN,NaT,NaT,2023-09-14
31827,39412,306231,336557.0,243846.0,2022-03-09 14:10:51.217884,NaT,None,None,243704.0,NaN,...,a5t3b000000AGJJAA4,2023-05-10 20:55:37.981692,{other},False,None,None,NaN,NaT,NaT,2023-06-16
31828,43410,359825,NaN,NaN,2023-02-17 13:02:39.892716,NaT,None,None,249112.0,NaN,...,a5t3b000001ROP5AAO,NaT,{en-us},False,None,None,NaN,NaT,NaT,2023-06-30
31829,44800,410799,NaN,NaN,2023-06-15 15:13:47.660358,NaT,None,None,NaN,NaN,...,a5t3b000001WfIHAA0,NaT,{en-us},False,None,None,NaN,NaT,NaT,2023-11-20


## write parquet files to aws

In [6]:
# import boto3
# import awswrangler as wr

In [7]:
# session = boto3.Session(
#     aws_access_key_id=ACCESS_KEY,
#     aws_secret_access_key=SECRET_KEY,
#     region_name="us-east-1"
# )

# wr.s3.to_parquet(
#     df=df,
#     path='s3://b-data-stg/1-raw/assessments/reviews-parquet/',
#     boto3_session=session,
#     dataset=True
# )

## great expectations use case

In [10]:
import great_expectations as ge

In [18]:
ge_df = ge.from_pandas(
    df
)

In [19]:
test = ge_df.expect_column_values_to_be_between("cost_plus_points", 0, 4)

In [23]:
test_2 = ge_df.expect_column_values_to_not_be_null("survey_sent_at")

In [25]:
tests = [ge_df.expect_column_values_to_be_between("cost_plus_points", 0, 4), 
         ge_df.expect_column_values_to_not_be_null("survey_sent_at")]

In [26]:
results = [t["success"] for t in tests]

In [27]:
if False in results:
    print("aaaa")

aaaa


In [110]:
ge_df.get_expectation_suite()

{
  "expectation_suite_name": "default",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_values_to_be_between",
      "kwargs": {
        "column": "cost_plus_points",
        "min_value": 0,
        "max_value": 4
      },
      "meta": {}
    }
  ],
  "data_asset_type": "Dataset",
  "meta": {
    "great_expectations_version": "0.18.5"
  }
}